# qUnits Demo

In [1]:
import qrobot
import time

First, define a model and a desired burst:

In [2]:
qrobot.models.AngularModel(n=2, tau=10)

[model: AngularModel, n: 2, tau: 10]

In [3]:
qrobot.bursts.ZeroBurst()

You can use objects like those to create a basic qBrain:

<img width=200 src="./tutorial_qunits_basicnetwork.png"/>

In [4]:
from qrobot.models import AngularModel
from qrobot.bursts import ZeroBurst

# Layer 0
l0_unit0 = qrobot.qunits.QUnit(
    name="l0_unit0",
    model=AngularModel(n=2, tau=10),
    burst=ZeroBurst(),
    Ts=0.1,
    query=[0.1, 0.5],  # Query target initialized
    # No input
)

# Layer 1
l1_unit0 = qrobot.qunits.QUnit(
    name="l1_unit0",
    model=AngularModel(n=1, tau=3),
    burst=ZeroBurst(),
    Ts=1,
    in_qunits={0: l0_unit0.id},  # Will receive Input from l0_unit0, dim 0
)

l1_unit1 = qrobot.qunits.QUnit(
    name="l1_unit1",
    model=AngularModel(n=1, tau=5),
    burst=ZeroBurst(),
    Ts=1,
    in_qunits={0: l0_unit0.id},  # Will receive input from l0_unit0, dim 1
)

In [5]:
l0_unit0

QUnit "l0_unit0-6c6eb7"
     name:	l0_unit0
     id:	l0_unit0-6c6eb7
     model:	[model: AngularModel, n: 2, tau: 10]
     burst:	<class 'qrobot.bursts.zeroburst.ZeroBurst'>
     query:	[0.1, 0.5]
     Ts:	0.1

In [6]:
l1_unit0

QUnit "l1_unit0-dd3ade"
     name:	l1_unit0
     id:	l1_unit0-dd3ade
     model:	[model: AngularModel, n: 1, tau: 3]
     burst:	<class 'qrobot.bursts.zeroburst.ZeroBurst'>
     query:	[0.0]
     Ts:	1.0

In [7]:
l1_unit1

QUnit "l1_unit1-e220ed"
     name:	l1_unit1
     id:	l1_unit1-e220ed
     model:	[model: AngularModel, n: 1, tau: 5]
     burst:	<class 'qrobot.bursts.zeroburst.ZeroBurst'>
     query:	[0.0]
     Ts:	1.0

The input units for each qUnit are:

In [8]:
print(l0_unit0.in_qunits)
print(l1_unit0.in_qunits)
print(l1_unit1.in_qunits)

{0: None, 1: None}
{0: 'l0_unit0-6c6eb7'}
{0: 'l0_unit0-6c6eb7'}


Add an external input for `l0_unit0`:

In [9]:
l0_unit0.input_vector

[0.0, 0.0]

In [10]:
l0_unit0.input_vector = [0.5, 0.7]

2022-09-11 23:03:22,166 — l0_unit0-6c6eb7 — WARNING — input_vector:186 — The qunit.input_vector setter will be deprecated with the introduction of sensorial interfaces, since it should not be possible to hard-write qUnits' inputs from the python code directly.


In [11]:
l0_unit0.input_vector

[0.5, 0.7]

In [12]:
l0_unit0.start()
l1_unit0.start()
l1_unit1.start()

2022-09-11 23:03:30,113 — l0_unit0-6c6eb7 — INFO — start:241 — Starting qUnit
2022-09-11 23:03:30,153 — l1_unit0-dd3ade — INFO — start:241 — Starting qUnit
2022-09-11 23:03:30,191 — l1_unit1-e220ed — INFO — start:241 — Starting qUnit
2022-09-11 23:03:30,197 — l1_unit0-dd3ade — INFO — input_vector:166 — Unable to read l0_unit0-6c6eb7 input
2022-09-11 23:03:30,240 — l1_unit1-e220ed — INFO — input_vector:166 — Unable to read l0_unit0-6c6eb7 input
2022-09-11 23:03:31,215 — l1_unit0-dd3ade — INFO — input_vector:166 — Unable to read l0_unit0-6c6eb7 input
2022-09-11 23:03:31,260 — l1_unit1-e220ed — INFO — input_vector:166 — Unable to read l0_unit0-6c6eb7 input


Visualize the time evolution of the system from the redis status for 30 seconds:

In [13]:
import time
import json
from IPython.display import clear_output

for _ in range(30):
    clear_output(wait=True)
    status = qrobot.qunits.redis_utils.redis_status()
    print(json.dumps(status, indent=1, sort_keys=True))
    time.sleep(1)

{
 "l0_unit0-6c6eb7": "0.5",
 "l0_unit0-6c6eb7 state": "10",
 "l1_unit0-dd3ade": "1.0",
 "l1_unit0-dd3ade state": "0",
 "l1_unit1-e220ed": "1.0",
 "l1_unit1-e220ed state": "0"
}


<img width=200 src="./tutorial_qunits_basicnetwork.png"/>

Stop the processing loops:

In [14]:
l0_unit0.stop()
l1_unit0.stop()
l1_unit1.stop()

2022-09-11 23:04:05,611 — l0_unit0-6c6eb7 — INFO — stop:249 — Stopping qUnit
2022-09-11 23:04:05,620 — l1_unit0-dd3ade — INFO — stop:249 — Stopping qUnit
2022-09-11 23:04:05,626 — l1_unit1-e220ed — INFO — stop:249 — Stopping qUnit


Flush the redis to clean all traces (should not be necessary if the qUnits processing loops stopped correctly):

In [15]:
qrobot.qunits.redis_utils.redis_status()

{}

In [16]:
qrobot.qunits.redis_utils.flush_redis()

2022-09-11 23:04:07,655 — redis — INFO — flush_redis:81 — Flushing redis database


Print the last 30 lines of the log:

In [17]:
log_file = qrobot._logger.log_file()
n_lines = 30

with open(log_file) as log:
    for line in (log.readlines() [-n_lines:]):
        print(line, end ='')

2022-09-11 23:04:04,657 — l1_unit0-dd3ade — DEBUG — _loop:300 — input_vector=[0.5]
2022-09-11 23:04:04,686 — l0_unit0-6c6eb7 — DEBUG — _loop:296 — Temporal window event 5/10
2022-09-11 23:04:04,687 — l0_unit0-6c6eb7 — DEBUG — _loop:300 — input_vector=[0.5, 0.7]
2022-09-11 23:04:04,789 — l0_unit0-6c6eb7 — DEBUG — _loop:296 — Temporal window event 6/10
2022-09-11 23:04:04,790 — l0_unit0-6c6eb7 — DEBUG — _loop:300 — input_vector=[0.5, 0.7]
2022-09-11 23:04:04,891 — l0_unit0-6c6eb7 — DEBUG — _loop:296 — Temporal window event 7/10
2022-09-11 23:04:04,892 — l0_unit0-6c6eb7 — DEBUG — _loop:300 — input_vector=[0.5, 0.7]
2022-09-11 23:04:04,993 — l0_unit0-6c6eb7 — DEBUG — _loop:296 — Temporal window event 8/10
2022-09-11 23:04:04,994 — l0_unit0-6c6eb7 — DEBUG — _loop:300 — input_vector=[0.5, 0.7]
2022-09-11 23:04:05,095 — l0_unit0-6c6eb7 — DEBUG — _loop:296 — Temporal window event 9/10
2022-09-11 23:04:05,096 — l0_unit0-6c6eb7 — DEBUG — _loop:300 — input_vector=[0.5, 0.7]
2022-09-11 23:04:05,19